In [1]:
import numpy as np 
import pandas as pd 
import scipy.stats as st  
import yfinance as yf 
import datetime as dt 
from _measurements import * 
from _distribution_test import * 

import seaborn as sns 
import matplotlib.pyplot as plt

sns.set_theme()

In [2]:
# List of symbols
symbols = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'JNJ', 'WMT', 'KO', 'BAC', 'ADBE', 'CSCO', 'MCD', 'NFLX', 'NKE', 'PUM.DE',
    'PM', 'IBM', 'GE'
]

# Download historical data for the symbols
data = yf.download(symbols, start='2007-01-01', end='2023-08-07')["Close"]

# Print the first few rows of the data
data.dropna(inplace=True)
returns = data.pct_change().dropna()
returns = returns["AAPL"]

[*********************100%***********************]  18 of 18 completed


In [3]:
st = Measurements(returns).analyze(returns, pd.DataFrame(np.ones(returns.shape[0])/returns.shape[0]))

In [4]:
pd.DataFrame(st)

,max_drawdown,volatility,sharpe_ratio,kurtosis,skewness,var_hist,cvar_hist,var_gauss,cvar_gauss,var_nongauss,cvar_nongauss
0,0.588334,0.315521,0.058837,6.27345,-0.099118,0.029497,0.001419,0.031519,0.001217,0.02956,0.00141
